In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install google

In [ ]:
import requests
import csv
import pandas as pd
import random
from bs4 import BeautifulSoup
from tqdm import tqdm
from googlesearch import search
import time

In [ ]:
# Definir múltiples consultas con palabras clave
keywords = ["violencia", "salud", "economía", "seguridad", "elecciones", "corrupción", "pandemia"]
base_query = "site:animalpolitico.com/verificacion-de-hechos/fact-checking"

# Usar un conjunto para evitar enlaces duplicados
news_links = set()

# Realizar búsquedas con diferentes palabras clave
try:
    for keyword in keywords:
        query = f"{base_query} {keyword}"
        print(f"\n🔎 Buscando: {query}")

        for result in search(query, num_results=100):
            if "/verificacion-de-hechos/fact-checking/" in result:
                news_links.add(result)
            time.sleep(2)  # Pausa para evitar bloqueos de Google

except Exception as e:
    print("Error en la búsqueda:", e)

# Guardar en un archivo CSV
csv_filename = "/content/sample_data/news_el_sabueso/noticias_animal_politico_ampliado.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["URL"])  # Encabezado
    for link in news_links:
        writer.writerow([link])

print(f"\nTotal de artículos guardados: {len(news_links)}")
print(f"📂 Los enlaces se guardaron en {csv_filename}")


🔎 Buscando: site:animalpolitico.com/verificacion-de-hechos/fact-checking violencia

🔎 Buscando: site:animalpolitico.com/verificacion-de-hechos/fact-checking salud

🔎 Buscando: site:animalpolitico.com/verificacion-de-hechos/fact-checking economía

🔎 Buscando: site:animalpolitico.com/verificacion-de-hechos/fact-checking seguridad

🔎 Buscando: site:animalpolitico.com/verificacion-de-hechos/fact-checking elecciones

🔎 Buscando: site:animalpolitico.com/verificacion-de-hechos/fact-checking corrupción

🔎 Buscando: site:animalpolitico.com/verificacion-de-hechos/fact-checking pandemia

Total de artículos guardados: 505
📂 Los enlaces se guardaron en /content/sample_data/news_el_sabueso/noticias_animal_politico_ampliado.csv


In [ ]:
# Define the CSV file path
csv_filename = "/content/sample_data/news_el_sabueso/noticias_animal_politico_ampliado.csv"
cleaned_csv_filename = "/content/sample_data/news_el_sabueso/noticias_animal_politico_limpio.csv"

# Load the CSV file
df = pd.read_csv(csv_filename)

# Remove duplicates
df_cleaned = df.drop_duplicates()

# Save the cleaned CSV file
df_cleaned.to_csv(cleaned_csv_filename, index=False)

print(f"✅ CSV cleaned! Original: {len(df)} rows → Cleaned: {len(df_cleaned)} rows")
print(f"📂 Cleaned file saved as: {cleaned_csv_filename}")

✅ CSV cleaned! Original: 505 rows → Cleaned: 505 rows
📂 Cleaned file saved as: /content/sample_data/news_el_sabueso/noticias_animal_politico_limpio.csv


In [ ]:
df_cleaned

,URL
0,https://animalpolitico.com/verificacion-de-hec...
1,https://animalpolitico.com/verificacion-de-hec...
2,https://animalpolitico.com/verificacion-de-hec...
3,https://animalpolitico.com/verificacion-de-hec...
4,https://animalpolitico.com/verificacion-de-hec...
...,...
500,https://animalpolitico.com/verificacion-de-hec...
501,https://animalpolitico.com/verificacion-de-hec...
502,https://animalpolitico.com/verificacion-de-hec...
503,https://animalpolitico.com/verificacion-de-hec...


In [ ]:
!wget -q -O google-chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome.deb
!apt-get install -f
!rm google-chrome.deb

!wget -q -O chromedriver.zip https://storage.googleapis.com/chrome-for-testing-public/133.0.6943.141/linux64/chromedriver-linux64.zip
!unzip -q chromedriver.zip
!chmod +x chromedriver
!mv chromedriver /usr/local/bin/

Selecting previously unselected package google-chrome-stable.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack google-chrome.deb ...
Unpacking google-chrome-stable (133.0.6943.141-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is not installed.

dpkg: error processing package google-chrome-stable (--install):
 dependency problems - leaving unconfigured
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Errors were encountered while processing:
 google-chrome-stable
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Correcting dependencies... Done
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  libvulkan1 mesa-vulkan-drivers
0 upgraded, 2 newly installed, 0 t

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 33.9 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def scrape_animal_politico(urls, headless=True, retries=3):
    """
    Scrape fact-checking articles from Animal Político.

    Parameters:
    - urls (list): List of URLs to scrape.
    - headless (bool): Whether to run Chrome in headless mode.
    - retries (int): Number of retries for failed extractions.

    Returns:
    - DataFrame with extracted data.
    """

    # Configurar ChromeOptions
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.binary_location = "/usr/bin/google-chrome"

    # Inicializar WebDriver
    service = Service("/usr/local/bin/chromedriver")  # Asegúrate de tener el path correcto
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Lista para almacenar los datos extraídos
    data = []

    for url in urls:
        print(f"\n🔍 Scraping: {url}")

        for attempt in range(retries):
            try:
                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "h1"))
                )  # Esperar hasta que cargue el título

                # Extraer título
                try:
                    title = driver.find_element(By.TAG_NAME, "h1").text.strip()
                except:
                    title = "No encontrado"

                # Extraer fecha y autor
                try:
                    date_author_section = driver.find_element(By.XPATH,
                        "//div[contains(@class, 'flex') and contains(@class, 'justify-between') and contains(@class, 'text-[15px]')]"
                    ).text.strip()
                    date_author = date_author_section.split("\n")
                    date = date_author[0].strip() if len(date_author) > 0 else "No encontrado"
                    author = date_author[1].strip() if len(date_author) > 1 else "No encontrado"
                except:
                    date, author = "No encontrado", "No encontrado"

                # Extraer categoría desde imagen (badge)
                try:
                    categoria_img = driver.find_element(By.XPATH,
                        "//div[contains(@class, 'relative z-[5]')]//img"
                    ).get_attribute("alt")
                    categoria = categoria_img.replace("badge-", "").capitalize() if categoria_img else "No encontrado"
                except:
                    categoria = "No encontrado"

                # Extraer el cuadro de verificación
                try:
                    cuadro_verificacion = driver.find_element(By.XPATH,
                        "//div[contains(@class, 'bg-[#f3f3f2]')]"
                    ).text.strip()
                except:
                    cuadro_verificacion = "No encontrado"

                # Extraer contenido del artículo
                try:
                    paragraphs = driver.find_elements(By.CSS_SELECTOR, "div.post-details p")
                    content = "\n".join([p.text.strip() for p in paragraphs])
                except:
                    content = "No encontrado"

                # Almacenar los datos extraídos
                data.append([title, categoria, date, author, cuadro_verificacion, content[:500], url])
                print(f"✅ Scrapeo exitoso para: {url}")
                break  # Salir del loop de retries si fue exitoso

            except Exception as e:
                print(f"⚠️ Intento {attempt+1} fallido para {url}: {e}")
                time.sleep(3)  # Espera antes de reintentar

                if attempt == retries - 1:
                    print(f"❌ No se pudo extraer la información de {url}")

    driver.quit()

    # Convertir los datos en DataFrame
    df = pd.DataFrame(data, columns=["Título", "Categoría", "Fecha", "Autor", "Cuadro de Verificación", "Contenido", "URL"])

    return df


In [ ]:
# Define the path to the cleaned CSV file
cleaned_csv_filename = "/content/sample_data/news_el_sabueso/noticias_animal_politico_limpio.csv"

# Load the CSV file and extract URLs into a list
df = pd.read_csv(cleaned_csv_filename)
urls = df["URL"].dropna().tolist()  # Drop empty values and convert to list

# Print the first 5 URLs to verify
print("Example URLs:", urls[:5])



Example URLs: ['https://animalpolitico.com/verificacion-de-hechos/fact-checking/falso-bajo-migracion-eu-amlo?rtbref=rtb_dhxxgitjzqqgwggjwamv_1713896813407', 'https://animalpolitico.com/verificacion-de-hechos/fact-checking/candidatos-puebla-presumen-logros-enganosos?rtbref=rtb_r4a8z8t0fxdakgs5w1h7_1714247702255', 'https://animalpolitico.com/verificacion-de-hechos/fact-checking/rubalcava-remplazo-tuberias-enganoso', 'https://animalpolitico.com/verificacion-de-hechos/fact-checking/dichos-falsos-amlo-quinto-informe', 'https://animalpolitico.com/verificacion-de-hechos/fact-checking/spots-cuarto-informe-de-amlo-dichos-enganosos-y-verdaderos']


In [ ]:
df_scrape = scrape_animal_politico(urls)
df_scrape.to_csv("/content/sample_data/news_el_sabueso/fact_checking_results.csv")


🔍 Scraping: https://animalpolitico.com/verificacion-de-hechos/fact-checking/falso-bajo-migracion-eu-amlo?rtbref=rtb_dhxxgitjzqqgwggjwamv_1713896813407
✅ Scrapeo exitoso para: https://animalpolitico.com/verificacion-de-hechos/fact-checking/falso-bajo-migracion-eu-amlo?rtbref=rtb_dhxxgitjzqqgwggjwamv_1713896813407

🔍 Scraping: https://animalpolitico.com/verificacion-de-hechos/fact-checking/candidatos-puebla-presumen-logros-enganosos?rtbref=rtb_r4a8z8t0fxdakgs5w1h7_1714247702255
✅ Scrapeo exitoso para: https://animalpolitico.com/verificacion-de-hechos/fact-checking/candidatos-puebla-presumen-logros-enganosos?rtbref=rtb_r4a8z8t0fxdakgs5w1h7_1714247702255

🔍 Scraping: https://animalpolitico.com/verificacion-de-hechos/fact-checking/rubalcava-remplazo-tuberias-enganoso
✅ Scrapeo exitoso para: https://animalpolitico.com/verificacion-de-hechos/fact-checking/rubalcava-remplazo-tuberias-enganoso

🔍 Scraping: https://animalpolitico.com/verificacion-de-hechos/fact-checking/dichos-falsos-amlo-quint

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
import pandas as pd

In [18]:
# Define custom headers
headers = ["Título", "Categoría", "Contenido", "URL", "Time"]

# Read CSV and assign headers
df_scrape = pd.read_csv("/content/drive/MyDrive/MCD Folder/2025-1/AAA/Proyecto/fact_checking_results_parallel.csv", names=headers, header=None)

# Display the first few rows
df_scrape.head()

,Título,Categoría,Contenido,URL,Time
0,Candidatos en Puebla presumen logros y transpa...,Enganoso,Los candidatos al gobierno de Puebla Eduardo R...,https://animalpolitico.com/verificacion-de-hec...,77.89
1,Adrián Rubalcava promete como candidato al Sen...,Enganoso,El candidato a senador suplente por el Partido...,https://animalpolitico.com/verificacion-de-hec...,78.27
2,Spots del cuarto informe de AMLO: Los dichos e...,Enganoso,"A unos días del Cuarto Informe de Gobierno, ya...",https://animalpolitico.com/verificacion-de-hec...,85.47
3,AMLO dice que “el país está en paz” en víspera...,Falso,El presidente Andrés Manuel López Obrador afir...,https://animalpolitico.com/verificacion-de-hec...,57.87
4,El Sabueso: Disminuir la violencia en Tamaulip...,Falso,Frase a revisión: “Vamos a promover un ambient...,https://animalpolitico.com/verificacion-de-hec...,15.09


In [19]:
df_scrape.duplicated().sum()

505

In [20]:
df_scrape["content_length"] = df_scrape["Contenido"].str.len()
df_cleaned = df_scrape.sort_values(by="content_length", ascending=False).drop_duplicates(subset=["URL"], keep="first")
df_cleaned = df_cleaned.drop(columns=["content_length"])



In [21]:
df_cleaned.duplicated().sum()

0